# MorphMultiwordProperNamesCorrector

In [1]:
from estnltk import Text, Layer, Annotation

from estnltk.taggers import Retagger
from estnltk.taggers import VabamorfTagger
from estnltk.taggers import MorphAnalysisReorderer


class MorphMultiwordProperNamesCorrector(Retagger):
    """Reanalyses sentences that contain multiword propername phrases with VabamorfTagger."""
    conf_param = ('vabamorftagger', 'morph_reorderer')
    
    def __init__(self, output_layer='morph_analysis', input_layers=['morph_analysis', 'sentences']):
        self.output_layer = output_layer
        self.input_layers = input_layers
        self.output_attributes = VabamorfTagger.output_attributes
        self.vabamorftagger    = VabamorfTagger()
        self.morph_reorderer   = MorphAnalysisReorderer()

    def _change_layer(self, raw_text, layers, status):
        # Analyse input sentence by sentence
        morph_layer = layers[self.output_layer]
        morph_span_id = 0
        reanalysed_sentences = 0
        reanalysed_words     = 0
        for sentence in layers[ self.input_layers[1] ]:
            multiword_propernames_locations = set()
            last_partofspeech = None
            words_in_sentence = []
            local_word_id = 0
            while morph_span_id < len( morph_layer): 
                morph_span = morph_layer[morph_span_id]
                if sentence.start <= morph_span.start and \
                    morph_span.end <= sentence.end:
                    # a) word is in this sentence
                    if 'H' in morph_span.partofspeech and last_partofspeech == 'H':
                        # Remember locations of multiword propernames
                        multiword_propernames_locations.add( local_word_id - 1 )
                        multiword_propernames_locations.add( local_word_id )
                    words_in_sentence.append( morph_span )
                    morph_span_id += 1
                    local_word_id += 1
                    last_partofspeech = 'H' if 'H' in morph_span.partofspeech else morph_span.annotations[0]['partofspeech']
                elif sentence.end <= morph_span.start:
                    # b) word is in the next sentence
                    break
            # Reanalyse the sentence that contains multiword proper names
            if len(multiword_propernames_locations) > 0:
                # Make a new Text for sentence; add tokenization
                sentence_str = sentence.enclosing_text
                sentence_text = Text(sentence_str).tag_layer(['words', 'sentences'])
                # If everything is ok with tokenization, then reapply morph
                if len(sentence_text['words']) == len(words_in_sentence):
                    # Add morph + reordering
                    self.vabamorftagger.tag( sentence_text )
                    self.morph_reorderer.retag( sentence_text )
                    # Transfer new analyses to multiword propernames
                    for wid, old_morph_span in enumerate( words_in_sentence ):
                        if wid in multiword_propernames_locations:
                            new_morph_annotations = sentence_text['morph_analysis'][wid].annotations
                            old_morph_span.clear_annotations()
                            for annotation in new_morph_annotations:
                                new_annotation_dict = annotation.to_record(with_text=False)
                                del new_annotation_dict['start']
                                del new_annotation_dict['end']
                                old_morph_span.add_annotation( Annotation(old_morph_span, **new_annotation_dict) )
                            reanalysed_words += 1
                    reanalysed_sentences += 1
        # Record stats
        morph_layer.meta['reanalysed_words'] = reanalysed_words
        morph_layer.meta['reanalysed_sentences'] = reanalysed_sentences


## Example

In this example, MorphMultiwordProperNamesCorrector is applied on a simple morph_analysed Text.

(!) But in order to get the full effect, you should apply it after the text has been analysed with VabamorfCorpusTagger and corrected with MorphAnalysisReorderer -- it should be the final post-correction for the morph analysis.

In [2]:
t=Text('''
Vormi mängude ja materjali valiku eest lavastusele " Nähtamatu poiss " Haapsalu Noorte Huvikeskuse loovus- ja 
draamaring " Üks " juhendajad Anne Suislep ja Triin Reemann Energilise lastepärase huumori ja loo jutustamise
eest lavastuses " Ull Jaak ehk küll on häda kui pea on mäda " juhendaja Kadi Kronberg Lasva Rahvamaja laste 
näitering ( Võrumaa ). Preemia väljaandjaks on MTÜ Vilde Teater. Edgar Savisaar. Siin lauses pole midagi analüüsida. 
See on Tõlkebüroo Filosoof.''')
       
t.tag_layer()
t

Text(text='\nVormi mängude ja materjali valiku eest lavastusele " Nähtamatu poiss " Haapsalu Noorte Huvikeskuse loovus- ja \ndraamaring " Üks " juhendajad Anne Suislep ja Triin Reemann Energilise lastepärase huumori ja loo jutustamise\neest lavastuses " Ull Jaak ehk küll on häda kui pea on mäda " juhendaja Kadi Kronberg Lasva Rahvamaja laste \nnäitering ( Võrumaa ). Preemia väljaandjaks on MTÜ Vilde Teater. Edgar Savisaar. Siin lauses pole midagi analüüsida. \nSee on Tõlkebüroo Filosoof.')

In [3]:
corrector = MorphMultiwordProperNamesCorrector()
corrector.retag(t)
t.morph_analysis

Layer(name='morph_analysis', attributes=('normalized_text', 'lemma', 'root', 'root_tokens', 'ending', 'clitic', 'form', 'partofspeech'), spans=SL[Span('Vormi', [{'normalized_text': 'Vormi', 'lemma': 'vorm', 'root': 'vorm', 'root_tokens': ['vorm'], 'ending': '0', 'clitic': '', 'form': 'sg g', 'partofspeech': 'S'}]),
Span('mängude', [{'normalized_text': 'mängude', 'lemma': 'mäng', 'root': 'mäng', 'root_tokens': ['mäng'], 'ending': 'de', 'clitic': '', 'form': 'pl g', 'partofspeech': 'S'}]),
Span('ja', [{'normalized_text': 'ja', 'lemma': 'ja', 'root': 'ja', 'root_tokens': ['ja'], 'ending': '0', 'clitic': '', 'form': '', 'partofspeech': 'J'}]),
Span('materjali', [{'normalized_text': 'materjali', 'lemma': 'materjal', 'root': 'materjal', 'root_tokens': ['materjal'], 'ending': '0', 'clitic': '', 'form': 'sg g', 'partofspeech': 'S'}]),
Span('valiku', [{'normalized_text': 'valiku', 'lemma': 'valik', 'root': 'valik', 'root_tokens': ['valik'], 'ending': '0', 'clitic': '', 'form': 'sg g', 'partofspeech': 'S'}]),
Span('eest', [{'normalized_text': 'eest', 'lemma': 'eest', 'root': 'eest', 'root_tokens': ['eest'], 'ending': '0', 'clitic': '', 'form': '', 'partofspeech': 'K'}]),
Span('lavastusele', [{'normalized_text': 'lavastusele', 'lemma': 'lavastus', 'root': 'lavastus', 'root_tokens': ['lavastus'], 'ending': 'le', 'clitic': '', 'form': 'sg all', 'partofspeech': 'S'}]),
Span('"', [{'normalized_text': '"', 'lemma': '"', 'root': '"', 'root_tokens': ['"'], 'ending': '', 'clitic': '', 'form': '', 'partofspeech': 'Z'}]),
Span('Nähtamatu', [{'normalized_text': 'Nähtamatu', 'lemma': 'Nähtamatu', 'root': 'Nähtamatu', 'root_tokens': ['Nähtamatu'], 'ending': '0', 'clitic': '', 'form': 'sg g', 'partofspeech': 'H'}]),
Span('poiss', [{'normalized_text': 'poiss', 'lemma': 'poiss', 'root': 'poiss', 'root_tokens': ['poiss'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'S'}]),
Span('"', [{'normalized_text': '"', 'lemma': '"', 'root': '"', 'root_tokens': ['"'], 'ending': '', 'clitic': '', 'form': '', 'partofspeech': 'Z'}]),
Span('Haapsalu', [{'normalized_text': 'Haapsalu', 'lemma': 'Haapsalu', 'root': 'Haap_salu', 'root_tokens': ['Haap', 'salu'], 'ending': '0', 'clitic': '', 'form': 'sg g', 'partofspeech': 'H'}]),
Span('Noorte', [{'normalized_text': 'Noorte', 'lemma': 'noor', 'root': 'noor', 'root_tokens': ['noor'], 'ending': 'te', 'clitic': '', 'form': 'pl g', 'partofspeech': 'S'}]),
Span('Huvikeskuse', [{'normalized_text': 'Huvikeskuse', 'lemma': 'huvikeskus', 'root': 'huvi_keskus', 'root_tokens': ['huvi', 'keskus'], 'ending': '0', 'clitic': '', 'form': 'sg g', 'partofspeech': 'S'}]),
Span('loovus-', [{'normalized_text': 'loovus-', 'lemma': 'loovus', 'root': 'loovus', 'root_tokens': ['loovus'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'S'}]),
Span('ja', [{'normalized_text': 'ja', 'lemma': 'ja', 'root': 'ja', 'root_tokens': ['ja'], 'ending': '0', 'clitic': '', 'form': '', 'partofspeech': 'J'}]),
Span('draamaring', [{'normalized_text': 'draamaring', 'lemma': 'draamaring', 'root': 'draama_ring', 'root_tokens': ['draama', 'ring'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'S'}]),
Span('"', [{'normalized_text': '"', 'lemma': '"', 'root': '"', 'root_tokens': ['"'], 'ending': '', 'clitic': '', 'form': '', 'partofspeech': 'Z'}]),
Span('Üks', [{'normalized_text': 'Üks', 'lemma': 'üks', 'root': 'üks', 'root_tokens': ['üks'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'N'}, {'normalized_text': 'Üks', 'lemma': 'üks', 'root': 'üks', 'root_tokens': ['üks'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'P'}]),
Span('"', [{'normalized_text': '"', 'lemma': '"', 'root': '"', 'root_tokens': ['"'], 'ending': '', 'clitic': '', 'form': '', 'partofspeech': 'Z'}]),
Span('juhendajad', [{'normalized_text': 'juhendajad', 'lemma': 'juhendaja', 'root': 'juhendaja', 'root_tokens': ['juhendaja'], 'ending': 'd', 'clitic': '', 'form': 'pl n', 'partofspeech': 'S'}]),
Span('Anne', [{'normalized_text': '